In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The following block is used to divide the NTL csv into 10 csvs so that each iteration doesn't have to go through all the datapoints in CSV. Division is done on the basis of latitude range.

In [1]:
import os
import pandas as pd

# Load ntl.csv and llong.csv into DataFrames
ntl_data = pd.read_csv('/kaggle/input/uploadfiles/NTL data of haryana.csv')
llong_data = pd.read_csv('/kaggle/input/uploadfiles/merged_output.csv')

# Define a function to partition data and save to CSVs
def partition_and_save(data, column_name, output_prefix):
    min_val = data[column_name].min()
    max_val = data[column_name].max()
    interval = (max_val - min_val) / 10
    
    for i in range(10):
        lower_bound = min_val + i * interval
        upper_bound = min_val + (i + 1) * interval
        partitioned_data = data[(data[column_name] >= lower_bound) & (data[column_name] < upper_bound)]
        
        # Save the partitioned data to CSV in /kaggle/working/ directory
        output_path = f'/kaggle/working/{output_prefix}_part_{i}.csv'
        partitioned_data.to_csv(output_path, index=False)
        print(f'Saved {output_path}')

# Partition ntl_data based on latitude
partition_and_save(ntl_data, 'latitude', 'ntl')

# Partition llong_data based on Latitude
partition_and_save(llong_data, 'Latitude', 'llong')


Saved /kaggle/working/ntl_part_0.csv
Saved /kaggle/working/ntl_part_1.csv
Saved /kaggle/working/ntl_part_2.csv
Saved /kaggle/working/ntl_part_3.csv
Saved /kaggle/working/ntl_part_4.csv
Saved /kaggle/working/ntl_part_5.csv
Saved /kaggle/working/ntl_part_6.csv
Saved /kaggle/working/ntl_part_7.csv
Saved /kaggle/working/ntl_part_8.csv
Saved /kaggle/working/ntl_part_9.csv
Saved /kaggle/working/llong_part_0.csv
Saved /kaggle/working/llong_part_1.csv
Saved /kaggle/working/llong_part_2.csv
Saved /kaggle/working/llong_part_3.csv
Saved /kaggle/working/llong_part_4.csv
Saved /kaggle/working/llong_part_5.csv
Saved /kaggle/working/llong_part_6.csv
Saved /kaggle/working/llong_part_7.csv
Saved /kaggle/working/llong_part_8.csv
Saved /kaggle/working/llong_part_9.csv


The following code block calculates the closest lat-long in the NTL dataset and sets the NTL of this lat-long as the 'Closest NTL' of that patch.

In [2]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_0.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_0.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None
    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_0.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 25.0
Processed row 2 - Closest NTL: 26.0
Processed row 3 - Closest NTL: 25.0
Processed row 4 - Closest NTL: 25.0
Processed row 5 - Closest NTL: 26.0
Processed row 6 - Closest NTL: 25.0
Processed row 7 - Closest NTL: 25.0
Processed row 8 - Closest NTL: 26.0
Processed row 9 - Closest NTL: 25.0
Processed row 10 - Closest NTL: 32.0
Processed row 11 - Closest NTL: 30.0
Processed row 12 - Closest NTL: 30.0
Processed row 13 - Closest NTL: 32.0
Processed row 14 - Closest NTL: 30.0
Processed row 15 - Closest NTL: 30.0
Processed row 16 - Closest NTL: 32.0
Processed row 17 - Closest NTL: 30.0
Processed row 18 - Closest NTL: 30.0
Processed row 19 - Closest NTL: 34.0
Processed row 20 - Closest NTL: 34.0
Processed row 21 - Closest NTL: 35.0
Processed row 22 - Closest NTL: 34.0
Processed row 23 - Closest NTL: 34.0
Processed row 24 - Closest NTL: 35.0
Processed row 25 - Closest NTL: 32.0
Processed row 26 - Closest NTL: 34.0
Processed row 27 - Closest NTL: 33.0
Processed 

In [3]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_1.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_1.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None
    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_1.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 97.0
Processed row 2 - Closest NTL: 96.0
Processed row 3 - Closest NTL: 80.0
Processed row 4 - Closest NTL: 97.0
Processed row 5 - Closest NTL: 96.0
Processed row 6 - Closest NTL: 80.0
Processed row 7 - Closest NTL: 96.0
Processed row 8 - Closest NTL: 95.0
Processed row 9 - Closest NTL: 80.0
Processed row 10 - Closest NTL: 255.0
Processed row 11 - Closest NTL: 255.0
Processed row 12 - Closest NTL: 255.0
Processed row 13 - Closest NTL: 255.0
Processed row 14 - Closest NTL: 255.0
Processed row 15 - Closest NTL: 255.0
Processed row 16 - Closest NTL: 229.0
Processed row 17 - Closest NTL: 255.0
Processed row 18 - Closest NTL: 255.0
Processed row 19 - Closest NTL: 41.0
Processed row 20 - Closest NTL: 36.0
Processed row 21 - Closest NTL: 35.0
Processed row 22 - Closest NTL: 41.0
Processed row 23 - Closest NTL: 36.0
Processed row 24 - Closest NTL: 35.0
Processed row 25 - Closest NTL: 41.0
Processed row 26 - Closest NTL: 37.0
Processed row 27 - Closest NTL: 34.0
P

In [4]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_2.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_2.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None
    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_2.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 255.0
Processed row 2 - Closest NTL: 164.0
Processed row 3 - Closest NTL: 164.0
Processed row 4 - Closest NTL: 255.0
Processed row 5 - Closest NTL: 163.0
Processed row 6 - Closest NTL: 163.0
Processed row 7 - Closest NTL: 255.0
Processed row 8 - Closest NTL: 163.0
Processed row 9 - Closest NTL: 163.0
Processed row 10 - Closest NTL: 26.0
Processed row 11 - Closest NTL: 26.0
Processed row 12 - Closest NTL: 27.0
Processed row 13 - Closest NTL: 26.0
Processed row 14 - Closest NTL: 25.0
Processed row 15 - Closest NTL: 26.0
Processed row 16 - Closest NTL: 26.0
Processed row 17 - Closest NTL: 25.0
Processed row 18 - Closest NTL: 26.0
Processed row 19 - Closest NTL: 26.0
Processed row 20 - Closest NTL: 27.0
Processed row 21 - Closest NTL: 28.0
Processed row 22 - Closest NTL: 26.0
Processed row 23 - Closest NTL: 26.0
Processed row 24 - Closest NTL: 27.0
Processed row 25 - Closest NTL: 26.0
Processed row 26 - Closest NTL: 26.0
Processed row 27 - Closest NTL: 27.0
P

In [5]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_3.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_3.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None
    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_3.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 104.0
Processed row 2 - Closest NTL: 99.0
Processed row 3 - Closest NTL: 73.0
Processed row 4 - Closest NTL: 104.0
Processed row 5 - Closest NTL: 99.0
Processed row 6 - Closest NTL: 73.0
Processed row 7 - Closest NTL: 113.0
Processed row 8 - Closest NTL: 111.0
Processed row 9 - Closest NTL: 76.0
Processed row 10 - Closest NTL: 28.0
Processed row 11 - Closest NTL: 26.0
Processed row 12 - Closest NTL: 24.0
Processed row 13 - Closest NTL: 26.0
Processed row 14 - Closest NTL: 23.0
Processed row 15 - Closest NTL: 23.0
Processed row 16 - Closest NTL: 26.0
Processed row 17 - Closest NTL: 23.0
Processed row 18 - Closest NTL: 23.0
Processed row 19 - Closest NTL: 255.0
Processed row 20 - Closest NTL: 255.0
Processed row 21 - Closest NTL: 255.0
Processed row 22 - Closest NTL: 255.0
Processed row 23 - Closest NTL: 255.0
Processed row 24 - Closest NTL: 255.0
Processed row 25 - Closest NTL: 255.0
Processed row 26 - Closest NTL: 255.0
Processed row 27 - Closest NTL: 255

In [6]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_4.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_4.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None
    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_4.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 255.0
Processed row 2 - Closest NTL: 255.0
Processed row 3 - Closest NTL: 37.0
Processed row 4 - Closest NTL: 255.0
Processed row 5 - Closest NTL: 255.0
Processed row 6 - Closest NTL: 37.0
Processed row 7 - Closest NTL: 255.0
Processed row 8 - Closest NTL: 255.0
Processed row 9 - Closest NTL: 35.0
Processed row 10 - Closest NTL: 255.0
Processed row 11 - Closest NTL: 255.0
Processed row 12 - Closest NTL: 124.0
Processed row 13 - Closest NTL: 181.0
Processed row 14 - Closest NTL: 179.0
Processed row 15 - Closest NTL: 98.0
Processed row 16 - Closest NTL: 178.0
Processed row 17 - Closest NTL: 178.0
Processed row 18 - Closest NTL: 97.0
Processed row 19 - Closest NTL: 21.0
Processed row 20 - Closest NTL: 21.0
Processed row 21 - Closest NTL: 23.0
Processed row 22 - Closest NTL: 21.0
Processed row 23 - Closest NTL: 21.0
Processed row 24 - Closest NTL: 23.0
Processed row 25 - Closest NTL: 21.0
Processed row 26 - Closest NTL: 21.0
Processed row 27 - Closest NTL: 23

In [7]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_5.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_5.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None
    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_5.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 40.0
Processed row 2 - Closest NTL: 37.0
Processed row 3 - Closest NTL: 37.0
Processed row 4 - Closest NTL: 40.0
Processed row 5 - Closest NTL: 38.0
Processed row 6 - Closest NTL: 38.0
Processed row 7 - Closest NTL: 40.0
Processed row 8 - Closest NTL: 38.0
Processed row 9 - Closest NTL: 38.0
Processed row 10 - Closest NTL: 53.0
Processed row 11 - Closest NTL: 52.0
Processed row 12 - Closest NTL: 48.0
Processed row 13 - Closest NTL: 53.0
Processed row 14 - Closest NTL: 52.0
Processed row 15 - Closest NTL: 48.0
Processed row 16 - Closest NTL: 52.0
Processed row 17 - Closest NTL: 51.0
Processed row 18 - Closest NTL: 45.0
Processed row 19 - Closest NTL: 144.0
Processed row 20 - Closest NTL: 144.0
Processed row 21 - Closest NTL: 255.0
Processed row 22 - Closest NTL: 144.0
Processed row 23 - Closest NTL: 144.0
Processed row 24 - Closest NTL: 255.0
Processed row 25 - Closest NTL: 143.0
Processed row 26 - Closest NTL: 143.0
Processed row 27 - Closest NTL: 255.0
P

In [2]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_6.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_6.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None

    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_6.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 243.0
Processed row 2 - Closest NTL: 243.0
Processed row 3 - Closest NTL: 255.0
Processed row 4 - Closest NTL: 243.0
Processed row 5 - Closest NTL: 243.0
Processed row 6 - Closest NTL: 255.0
Processed row 7 - Closest NTL: 191.0
Processed row 8 - Closest NTL: 192.0
Processed row 9 - Closest NTL: 225.0
Processed row 10 - Closest NTL: 129.0
Processed row 11 - Closest NTL: 129.0
Processed row 12 - Closest NTL: 25.0
Processed row 13 - Closest NTL: 126.0
Processed row 14 - Closest NTL: 128.0
Processed row 15 - Closest NTL: 24.0
Processed row 16 - Closest NTL: 126.0
Processed row 17 - Closest NTL: 128.0
Processed row 18 - Closest NTL: 24.0
Processed row 19 - Closest NTL: 97.0
Processed row 20 - Closest NTL: 33.0
Processed row 21 - Closest NTL: 31.0
Processed row 22 - Closest NTL: 97.0
Processed row 23 - Closest NTL: 33.0
Processed row 24 - Closest NTL: 31.0
Processed row 25 - Closest NTL: 40.0
Processed row 26 - Closest NTL: 39.0
Processed row 27 - Closest NTL: 

In [3]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_7.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_7.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None
    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_7.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 24.0
Processed row 2 - Closest NTL: 25.0
Processed row 3 - Closest NTL: 25.0
Processed row 4 - Closest NTL: 25.0
Processed row 5 - Closest NTL: 25.0
Processed row 6 - Closest NTL: 25.0
Processed row 7 - Closest NTL: 24.0
Processed row 8 - Closest NTL: 22.0
Processed row 9 - Closest NTL: 22.0
Processed row 10 - Closest NTL: 171.0
Processed row 11 - Closest NTL: 57.0
Processed row 12 - Closest NTL: 56.0
Processed row 13 - Closest NTL: 83.0
Processed row 14 - Closest NTL: 182.0
Processed row 15 - Closest NTL: 181.0
Processed row 16 - Closest NTL: 83.0
Processed row 17 - Closest NTL: 182.0
Processed row 18 - Closest NTL: 181.0
Processed row 19 - Closest NTL: 109.0
Processed row 20 - Closest NTL: 108.0
Processed row 21 - Closest NTL: 26.0
Processed row 22 - Closest NTL: 109.0
Processed row 23 - Closest NTL: 108.0
Processed row 24 - Closest NTL: 26.0
Processed row 25 - Closest NTL: 29.0
Processed row 26 - Closest NTL: 28.0
Processed row 27 - Closest NTL: 27.0
P

In [4]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_8.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_8.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None
    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_8.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 93.0
Processed row 2 - Closest NTL: 95.0
Processed row 3 - Closest NTL: 79.0
Processed row 4 - Closest NTL: 93.0
Processed row 5 - Closest NTL: 95.0
Processed row 6 - Closest NTL: 79.0
Processed row 7 - Closest NTL: 94.0
Processed row 8 - Closest NTL: 96.0
Processed row 9 - Closest NTL: 82.0
Processed row 10 - Closest NTL: 227.0
Processed row 11 - Closest NTL: 227.0
Processed row 12 - Closest NTL: 255.0
Processed row 13 - Closest NTL: 227.0
Processed row 14 - Closest NTL: 227.0
Processed row 15 - Closest NTL: 255.0
Processed row 16 - Closest NTL: 118.0
Processed row 17 - Closest NTL: 120.0
Processed row 18 - Closest NTL: 154.0
Processed row 19 - Closest NTL: 46.0
Processed row 20 - Closest NTL: 46.0
Processed row 21 - Closest NTL: 42.0
Processed row 22 - Closest NTL: 108.0
Processed row 23 - Closest NTL: 110.0
Processed row 24 - Closest NTL: 109.0
Processed row 25 - Closest NTL: 106.0
Processed row 26 - Closest NTL: 109.0
Processed row 27 - Closest NTL: 1

In [5]:
import pandas as pd

# Load the CSV files into pandas DataFrames
ntl_data = pd.read_csv('/kaggle/working/ntl_part_9.csv')
merged_output = pd.read_csv('/kaggle/working/llong_part_9.csv')

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    dist=(lat1-lat2)**2 + (lon1-lon2)**2
    return dist

# Function to find the closest NTL for each image
def find_closest_ntl(row):
    min_distance = float('inf')
    closest_ntl = None
    
    for idx, ntl_row in ntl_data.iterrows():
        distance = calculate_distance(row['Latitude'], row['Longitude'],ntl_row['latitude'],ntl_row['longitude '])
        if distance < min_distance:
            min_distance = distance
            closest_ntl = ntl_row[' NTL']
    
    print(f"Processed row {row.name + 1} - Closest NTL: {closest_ntl}")
    return closest_ntl

# Apply the function to calculate closest NTL for each image
merged_output['Closest_NTL'] = merged_output.apply(find_closest_ntl, axis=1)

# Count the number of assigned closest NTLs
num_assigned_ntls = merged_output['Closest_NTL'].count()

# Save the updated DataFrame to a new CSV file in /kaggle/working/
output_path = '/kaggle/working/merged_output_with_ntl_9.csv'
merged_output.to_csv(output_path, index=False)

print(f"CSV file saved to: {output_path}")
print(f"Number of closest NTLs assigned: {num_assigned_ntls}")

Processed row 1 - Closest NTL: 255.0
Processed row 2 - Closest NTL: 255.0
Processed row 3 - Closest NTL: 255.0
Processed row 4 - Closest NTL: 255.0
Processed row 5 - Closest NTL: 255.0
Processed row 6 - Closest NTL: 255.0
Processed row 7 - Closest NTL: 255.0
Processed row 8 - Closest NTL: 255.0
Processed row 9 - Closest NTL: 255.0
Processed row 10 - Closest NTL: 255.0
Processed row 11 - Closest NTL: 255.0
Processed row 12 - Closest NTL: 255.0
Processed row 13 - Closest NTL: 255.0
Processed row 14 - Closest NTL: 255.0
Processed row 15 - Closest NTL: 255.0
Processed row 16 - Closest NTL: 255.0
Processed row 17 - Closest NTL: 255.0
Processed row 18 - Closest NTL: 255.0
Processed row 19 - Closest NTL: 10.0
Processed row 20 - Closest NTL: 11.0
Processed row 21 - Closest NTL: 11.0
Processed row 22 - Closest NTL: 10.0
Processed row 23 - Closest NTL: 11.0
Processed row 24 - Closest NTL: 11.0
Processed row 25 - Closest NTL: 10.0
Processed row 26 - Closest NTL: 11.0
Processed row 27 - Closest NT